In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import accuracy_score

# Book Recomentaton

In [ ]:
books=pd.read_csv('../input/goodreadsbooks/books.csv',error_bad_lines=False)
print(type(books))
books.head()

In [ ]:
books.tail()

In [ ]:
books.shape

In [ ]:
books.columns

In [ ]:
books.info()

In [ ]:
books.shape

Observations:

The data seems to have no missing values. There are total 11127 rows with total 12 columns with the range index from 0 to 11126.

There are 3 numerical columns with 3 int64 values and 0 float64 and 9 categorical (object) values. We can see that the column publication_date has the dtype as object.

We'll need to change that to datetime.

In the column names, we can see that num_pages is indented.

Overall the data looks in order. Memory usage: 1.0 MB

In [ ]:
#In the column names, we can see that num_pages is indented.remove it
books.rename(columns={'  num_pages': 'num_pages'},inplace=True)

In [ ]:
books.columns

In [ ]:
books.head()

In [ ]:
books.describe()

In [ ]:
#Any missing values?
books.isnull().sum().values.any()

In [ ]:
#Let's check the unique values
for i in books.columns:
    print('\n',i,'\n\n',books[i].unique())

In [ ]:
#Book with maximum rating count
print('\nBook with maximum rating count=',books['ratings_count'].max())
max_rating=books[books['ratings_count'] == books['ratings_count'].max()]
max_rating

Observations

Twilight by Stephenie Meyer published in 9/6/2006 by Little Brown and Company publisher has the highest ratings_count of 4597666.

Average_rating for this books is 3.5900 and text_review_counts is 94265

In [ ]:
# substring to be searched
sub ='J'
# seaching books by authors name 
#Series.str.find(sub, start=0, end=None)
books[books['authors'] == books['authors'].str.find(sub,start=0,end=None)]


Total number of books with highest and lowest average rating

In [ ]:

#Dividing numeric and categorical columns

numerical = books.select_dtypes(['float64','int64']).columns.values.tolist()
categorical = books.select_dtypes(['object']).columns.values.tolist()
print('Numerical Columns\n\n',numerical)
print()
print('Categorical Columns\n\n',categorical)


In [ ]:
books.head(3)

In [ ]:
#Changing data types of some columns like isbn into int64 
#average_rating (obj into float64)
books['average_rating']=pd.to_numeric(books['average_rating'],errors='coerce').fillna(0).astype(float)
books['isbn']=pd.to_numeric(books['isbn'],errors='coerce').fillna(0).astype(int)
books['isbn13']=pd.to_numeric(books['isbn13'],errors='coerce').fillna(0).astype(float)
books['num_pages']=pd.to_numeric(books['num_pages'],errors='coerce').fillna(0).astype(int)
books.info()

In [ ]:
books.describe()

In [ ]:
books.corr()

In [ ]:
corrMatrix=books.corr()
sns.heatmap(corrMatrix,annot=False,cmap='Blues')
plt.show()

In [ ]:
# books count by title
book_name = books.title.value_counts().to_frame().reset_index()
book_name.rename(columns={'title':'Count','index':'title'},inplace=True)
book_name.sort_values(by='Count',ascending=False)

In [ ]:
#top 10 most popular book
book_name_10=book_name.iloc[:10]
book_name_10

In [ ]:
plt.figure(figsize =[15,5])
sns.barplot(book_name_10.title,book_name_10.Count)
plt.title('Most popular titles of books',fontsize=14)
plt.xticks(rotation=25);

Observations

'The Iliad', 'The Brothers Karamazov', "A Midsummer Night's Dream", 'Anna Karenina', 'The Odyssey', "Gulliver's Travels", 'Collected Stories', 'The Great Gatsby', 'Robinson Crusoe', 'The Histories'

Are the top 10 popular titles.

# Books by author

In [ ]:
# book counts by authos
book_author = books['authors'].value_counts().to_frame().reset_index()
book_author.rename(columns={'authors':'Count','index':'Author'},inplace=True)
book_author.sort_values(by='Count',ascending=False)

In [ ]:
book_count=book_author[book_author['Count']>=30]
book_count

In [ ]:
plt.figure(figsize =[15,5])
sns.barplot(book_count.Author,book_count.Count)
plt.title('Some Most popular book Authors',fontsize=14)
plt.xticks(rotation=25);

Observations:-

Following authors have maximum number of books count  :-

Stephen King	40

P.G. Wodehouse	40

Rumiko Takahashi	39

Orson Scott Card	35

Agatha Christie	33

Piers Anthony	30



# Books by average_ratring

In [ ]:
#books min ad max average count
print('highest average rating = ',books['average_rating'].max())
print('lowest average rating =',books['average_rating'].min())

In [ ]:
#books details having maximum average_rating
avg_rat = books[books['average_rating'] == books['average_rating'].max()]
print('Total number of books with highest average rating = ',len(avg_rat))
avg_rat

In [ ]:
#Graphical represenation of books having maximum average rating
book_avg_count_5=avg_rat.iloc[:5]
plt.figure(figsize =[15,5])
sns.barplot(book_avg_count_5.title,book_avg_count_5.average_rating)
plt.title('titles of books which having maximum average rating',fontsize=14)
plt.xticks(rotation=90);


In [ ]:
#books details having mimimum average_rating
avg_rat_min= books[books['average_rating'] == books['average_rating'].min()]
print('Total number of books with highest average rating = ',len(avg_rat_min))
avg_rat_min[:10]

Observation:-

    There are 22 books having maximum average rating i.e 5. 
   
Books having maximum average ratings are:-

      '624      Comoediae 1: Acharenses/Equites/Nubes/Vespae/P...
786                      Willem de Kooning: Late Paintings
855      Literature Circle Guide: Bridge to Terabithia:...
1243     Middlesex Borough (Images of America: New Jersey)
4126     Zone of the Enders: The 2nd Runner Official St...
4790     The Diamond Color Meditation: Color Pathway to...
4935     Bulgakov's the Master and Margarita: The Text ...
5025     The Complete Theory Fun Factory: Music Theory ...
5476     The Goon Show  Volume 4: My Knees Have Fallen ...
5478     The Goon Show  Volume 11: He's Fallen in the W...
5649                                   Winchester Shotguns
5650     Colossians and Philemon: A Critical and Exeget...
6187                             Taxation of Mineral Rents
6250                           The New Big Book of America
6778     Delwau Duon: Peintiadau Nicholas Evans = Symph...
8547          Fanning the Flame: Bible  Cross  and Mission
9286     Oliver Wendell Holmes in Paris: Medicine  Theo...
9328                    Tyrannosaurus Wrecks (Stanley  #1)
9724         The Irish Anatomist: A Study of Flann O'Brien
9851     The American Campaign: U.S. Presidential Campa...
9897     His Princess Devotional: A Royal Encounter Wit...
10266           Bill Gates: Computer Legend (Famous Lives)

    
And there are 29 books having minimum average rating i.e 0
   
Books having minimum average ratings are:-

      '265      Out to Eat London 2002 (Lonely Planet Out to Eat)
375                         Juiced Official Strategy Guide
987               Open City 6: The Only Woman He Ever Left
2532     How To Meditate: An Anthology Of Talks On Medi...
2533            Insights: Talks On The Nature Of Existence
2758                   Venac sonetnih venaca; Puževa srma
3348     Streetcar Suburbs: The Process of Growth in Bo...
3493     Brodie's notes on Aldous Huxley's brave new world
4242                                   American Film Guide
4678     The Man and the Author: John Milton: Twentieth...
4702                     The Tolkien Fan's Medieval Reader
5325     Canopy: A Work for Voice and Light in Harvard ...
5877                        Patriots (The Coming Collapse)
6383            Laguna  I Love You: The Best of "Our Town"
6561                     American Writers  Supplement VIII
6862                              The Road To War  1933 39
6880     I'll Be Home Before Midnight and I Won't Get P...
7147     Mythographi Graeci 1: Apollodori Bibliotheca  ...
7402                      Sclerotherapy and vein treatment
7576                              The Berlin Phenomenology
7637          Your Child and Jesus: A Family Activity Book
7800                     Beckett Football Card Price Guide
8979     Brown's Star Atlas: Showing All The Bright Sta...
9335         Random House Crossword Mega Omnibus  Volume 1
9441     Classroom Interactions as Cross-Cultural Encou...
10142                                        Day and Night
10501    The Fantastic Vampire: Studies in the Children...
10921    The Leadership Challenge: Skills for Taking Ch...
11031    Anthony Thwaite: In Conversation With Peter Da...'
        

# Recomentation

# 1.Books recomenataion by considering  books title.

In [ ]:
#recomenation by book title
books_title=books.drop(['authors','language_code','publication_date','publisher'],axis=1)

In [ ]:
books_users=books_title.pivot(index=None,columns='bookID',values='average_rating').fillna(0)
matrix_books_users=csr_matrix(books_users.values)
print(matrix_books_users)

In [ ]:
knn=NearestNeighbors(metric='cosine',algorithm='brute',n_neighbors=10)

In [ ]:
knn.fit(matrix_books_users)

In [ ]:
# function for book recommention by its title
def recommender(book_title,data, model,n_recommentation):
    model.fit(data)
    idx=process.extractOne(book_title,books_title['title'])[2]
    print("Books selected:-",books_title['title'][idx],'Index:',idx)
    print('Searching for book recommendation.....')
    distances,indices=model.kneighbors(data[idx],n_neighbors=n_recommentation)
    for i in indices:
        print(books_title['title'][i].where(i!=idx))

In [ ]:
#recomend book by considering book 'Bill Bryson's African Diary'
recommender("Bill Bryson's African Diary",matrix_books_users,knn,5)

In [ ]:
#recomend book by considering book 'Comoediae 1'
recommender("Comoediae 1",matrix_books_users,knn,5)

# 2.Books recomenataion by considering  books authors.

In [ ]:
books_recom=books.drop(['language_code','publication_date','publisher'],axis=1)

In [ ]:
books_author=books_recom.pivot(index=None,columns='bookID',values='average_rating').fillna(0)
matrix_books_author=csr_matrix(books_author.values)
print(matrix_books_author)

In [ ]:
knn=NearestNeighbors(metric='cosine',algorithm='brute',n_neighbors=10)

In [ ]:
knn.fit(matrix_books_author)

In [ ]:
# function for book recommention by its author
def recommend_by_authors(book_author,data, model,n_recommentation):
    model.fit(data)
    idx=process.extractOne(book_author,books_recom['authors'])[2]
    print("Books selected by its author:-",books_recom['authors'][idx],'Index:',idx)
    print('Searching for book recommendation .....')
    distances,indices=model.kneighbors(data[idx],n_neighbors=n_recommentation)
    for i in indices:
        print(books_recom['title'][i].where(i!=idx))

In [ ]:

recommend_by_authors("J.K. Rowling/Mary GrandPré",matrix_books_author,knn,5)

In [ ]:
recommend_by_authors("C.S. Lewis/Ana Falcão Bastos",matrix_books_author,knn,5)